In [25]:
from abc import ABC, abstractmethod
from typing import Union

In [26]:
class Observerble(ABC):
    def __init__(self, *args, **kwargs):
        self._observers_list = []
    
    def attach(self, observer: 'Observer') -> None:
        self._observers_list.append(observer)
    
    def detach(self, observer: 'Observer') -> None:
        self._observers_list.remove(observer)
    
    @abstractmethod
    def notify(self, *args, **kwargs) -> None:
        pass

In [27]:
class Observer(ABC):
    @abstractmethod
    def update(self, *args, **kwargs) -> None:
        pass

In [28]:
class Strategy(ABC):
    @abstractmethod
    def eval(self):
        pass

In [29]:
class BinaryStrategy(Strategy):
    def __init__(self, a, b):
        self.a = a
        self.b = b
    
    def _get_values(self):
        value1 = self.a
        if isinstance(value1, ReactNumber):
            value1 = value1.value
        value2 = self.b
        if isinstance(value2, ReactNumber):
            value2 = value2.value
        return value1, value2


class MultiplicationStrategy(BinaryStrategy):
    """
    Класс, наследующийся от BinaryStrategy, вычисляет произведение чисел
    """
    def eval(self):
        """
        Функция умножения значений

        :return: Возвращает произведение чисел
        :rtype: int
        """
        a, b = self._get_values()
        return a * b


class SummatoryStrategy(BinaryStrategy):
    """
    Класс, наследующийся от BinaryStrategy, вычисляет сумму чисел
    """
    def eval(self):
        """
        Функция сложения значений

        :return: Возвращает сумму значений
        :rtype: int
        """
        a, b = self._get_values()
        return a + b


class DifferenceStrategy(BinaryStrategy):
    """
    Класс, наследующийся от BinaryStrategy, вычисляет разность чисел
    """
    def eval(self):
        """
        Функция вычитания значений

        :return: Возвращает разницу между числами
        :rtype: int
        """
        a, b = self._get_values()
        return a - b

In [30]:
class ReactNumber(Observerble, Observer):
    def __init__(self, number: Union[int, float, Strategy]):
        super().__init__(number)
        self.value = number

    @property
    def value(self) -> Union[int, float]:
        if self._number is None and self._strategy is not None:
            self._number = self._strategy.eval()
        return self._number

    @value.setter
    def value(self, new_value: Union[int, float, Strategy]) -> None:
        self._number = None
        self._strategy = None
        if isinstance(new_value, Strategy):
            self._strategy = new_value
        else:
            self._number = new_value
        self.notify()

    def __str__(self) -> str:
        return str(self.value)

    def __repr__(self) -> str:
        return str(self)

    def update(self, *args, **kwargs) -> None:
        if self._strategy is not None:
            self._number = self._strategy.eval()
        self.notify()

    def notify(self) -> None:
        for observer in self._observers_list:
            if hasattr(observer, 'update'):
                observer.update(self)

    def __mul__(self, other) -> 'ReactNumber':
        """
         Функция умножения
        :param other: другой объект
        :return: возвращает новое число
        """
        new_number = ReactNumber(MultiplicationStrategy(self, other))
        self.attach(new_number)
        if isinstance(other, ReactNumber):
            other.attach(new_number)
        return new_number

    def __rmul__(self, other) -> 'ReactNumber':
        """
        Функция отражённое умножение
        :param other: другой объект
        :return: возвращает новое число
        """
        new_number = ReactNumber(MultiplicationStrategy(other, self))
        self.attach(new_number)
        if isinstance(other, ReactNumber):
            other.attach(new_number)
        return new_number

    def __add__(self, other) -> 'ReactNumber':
        """
        Функция сложения
        :param other: другой объект
        :return: возвращает новое число
        """
        new_number = ReactNumber(SummatoryStrategy(self, other))
        self.attach(new_number)
        if isinstance(other, ReactNumber):
            other.attach(new_number)
        return new_number

    def __radd__(self, other) -> 'ReactNumber':
        """
        Функция отражённое сложение
        :param other: другой объект
        :return: возвращает новое число
        """
        new_number = ReactNumber(SummatoryStrategy(other, self))
        self.attach(new_number)
        if isinstance(other, ReactNumber):
            other.attach(new_number)
        return new_number

    def __sub__(self, other) -> 'ReactNumber':
        """
        Функция вычитания
        :param other: другой объект
        :return: возвращает новое число
        """
        new_number = ReactNumber(DifferenceStrategy(self, other))
        self.attach(new_number)
        if isinstance(other, ReactNumber):
            other.attach(new_number)
        return new_number

    def __rsub__(self, other) -> 'ReactNumber':
        """
        Функция отражённое вычитание
        :param other: другой объект
        :return: возвращает новое число
        """
        new_number = ReactNumber(DifferenceStrategy(self, other))
        self.attach(new_number)
        if isinstance(other, ReactNumber):
            other.attach(new_number)
        return new_number * -1

In [31]:
x = ReactNumber(5)
y = ReactNumber(10)
x, y

(5, 10)

In [32]:
y.value = 20

In [33]:
result = x * y + 3 * y * y
result

1300

In [34]:
result, type(result)

(1300, __main__.ReactNumber)